In [6]:
import scipy
import numpy as np
import pandas as pd
import anndata
import scvi 
import scanpy as sc

In [125]:
Ann = sc.read_h5ad("./Data/exp0/sc_data_filtered.h5ad")
annotations=pd.read_csv("./Data/exp0/Ann.csv")

In [126]:
annotations.drop(columns="Unnamed: 0", inplace=True)


In [127]:
t20=[]
for i in range(20):
    t20.append(annotations.value_counts().index[i][0])

In [128]:
t10=t20[0:10]
t15=t20[0:15]

In [129]:
predicted=pd.read_csv("./Stereoscope/Outputs/output_exp0.csv")
target=pd.read_csv("./Data/exp0/target.csv")

In [130]:
predicted.drop(columns="Unnamed: 0", inplace=True)

In [131]:
target.drop(columns="Unnamed: 0", inplace=True)

In [132]:
predicted_20=predicted[t20]
predicted_10=predicted[t10]
predicted_15=predicted[t15]

In [133]:
target_t20=t20
for i in range(20):
    target_t20[i]="cell_abundances_"+target_t20[i]

In [134]:
target_20=target[target_t20]
target_15=target[target_t20[0:15]]
target_10=target[target_t20[0:10]]

In [135]:
predicted=predicted.to_numpy()
predicted_20=predicted_20.to_numpy()
predicted_15=predicted_15.to_numpy()
predicted_10=predicted_10.to_numpy()
target_20=target_20.to_numpy()
target_15=target_15.to_numpy()
target_10=target_10.to_numpy()
target=target.to_numpy()

## Pearson Correlation
Entire output:

In [136]:
num_celltypes=49
d = np.zeros(num_celltypes)

In [137]:
for i in range(0,num_celltypes):
    p=predicted[:,i]
    t=target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]


In [138]:
np.mean(d)

0.6820255721166141

Top 10:

In [139]:
num_celltypes=10
d = np.zeros(num_celltypes)

In [140]:
for i in range(0,num_celltypes):
    p=predicted_10[:,i]
    t=target_10[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]



In [142]:
np.mean(d)

0.572459549252842

Top 15:

In [143]:
num_celltypes=15
d = np.zeros(num_celltypes)

In [144]:
for i in range(0,num_celltypes):
    p=predicted_15[:,i]
    t=target_15[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]




In [145]:
np.mean(d)

0.5368788476492824

Top 20

In [149]:
num_celltypes=20
d = np.zeros(num_celltypes)

In [150]:
for i in range(0,num_celltypes):
    p=predicted_20[:,i]
    t=target_20[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]



In [151]:
np.mean(d)

0.6083303088287784

## Jenson Shannon

In [152]:
from scipy.spatial import distance

In [153]:
jns=distance.jensenshannon(predicted,target,axis=1)
np.mean(jns)

0.4282545067817724

Top 10:

In [156]:
jns=distance.jensenshannon(predicted_10,target_10,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

nan

Top 15:

Top 20:

## F1 Score, Precision, Recall

All

In [157]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [159]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [160]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds


In [161]:
for t in thresholds:
    Predicted_logical=(predicted>t).astype(int)
    Target_logical=(target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [162]:
d

,Precision,Recall,F1-score
0.100,0.590919,0.396304,0.474429
0.200,0.357143,0.046358,0.082063
0.300,0.200000,0.002933,0.005780
0.010,0.571113,0.769730,0.655711
0.001,0.518735,0.871973,0.650493


Top 10

In [166]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds

In [167]:
for t in thresholds:
    Predicted_logical=(predicted_10>t).astype(int)
    Target_logical=(target_10>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [168]:
d

,Precision,Recall,F1-score
0.100,0.277519,0.257925,0.267364
0.200,0.000000,0.000000,0.000000
0.300,0.000000,0.000000,0.000000
0.010,0.412959,0.790246,0.542450
0.001,0.365608,0.879662,0.516533


Top 15

In [169]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds

In [170]:
for t in thresholds:
    Predicted_logical=(predicted_15>t).astype(int)
    Target_logical=(target_15>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [171]:
d

,Precision,Recall,F1-score
0.100,0.327184,0.382955,0.352880
0.200,0.389610,0.197368,0.262009
0.300,0.200000,0.050000,0.080000
0.010,0.364413,0.780366,0.496822
0.001,0.339823,0.855401,0.486410


Top 20

In [ ]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds

In [172]:
for t in thresholds:
    Predicted_logical=(predicted_20>t).astype(int)
    Target_logical=(target_20>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [173]:
d

,Precision,Recall,F1-score
0.100,0.545309,0.558711,0.551929
0.200,0.395833,0.079332,0.132174
0.300,0.200000,0.010101,0.019231
0.010,0.405319,0.816183,0.541652
0.001,0.357980,0.878939,0.508752
